In [1]:
from termcolor import colored
from collections import Counter
import requests
from requests.auth import HTTPBasicAuth
import os
from dotenv import load_dotenv
import json
import pandas as pd
# ------------------------------------------------------------------------------


def load_env_variables():
    load_dotenv()
    username = os.getenv("ELOQUA_USERNAME")
    password = os.getenv("ELOQUA_PASSWORD")
    return username, password


def authenticate(username, password):
    auth = HTTPBasicAuth(username, password)
    return auth


def update_lookup(auth):
    def get_contact_id(c_email, auth):
        url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contacts?search=name="{c_email}"'
        response = requests.get(url, auth=auth)

        if response.status_code == 200:
            contacts = response.json()
            if contacts:
                return contacts['elements'][0]['id']

            else:
                return None
        else:
            print(
                f'Erro ao buscar ID do contato. Código de status: {response.status_code}')
            return None

    def update_contact(contact_id, c_email, nome_do_campo_value, auth):
        url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contact/{contact_id}'
        payload = {
            "type": "Contact",
            "id": contact_id,
            "emailAddress": c_email,
            "salesPerson": nome_do_campo_value
        }

        response = requests.put(url, json=payload, auth=auth)

        if response.status_code == 200:
            print('Informações do contato atualizadas com sucesso.')
        else:
            print(
                f'Erro ao atualizar informações do contato. Código de status: {response.status_code}')

    print("Verificando lookup...")

    nome_lookup_table = "FindLaw Rep to user ID"

    # Passo 1: Fazer solicitação à primeira API
    eloqua_url = "https://secure.p04.eloqua.com/API/REST/2.0/data/contact/view/100007/contacts/campaign/element/300040"
    response = requests.get(eloqua_url, auth=(auth))
    contact_data = response.json()

    # Verificar se a solicitação foi bem-sucedida
    if response.status_code != 200:
        print(
            f"Falha ao obter dados do contato. Código de status: {response.status_code}")
        print(f"Resposta: {response.text}")
        exit()

    # Iterar sobre os elementos (contatos)
    for contact in contact_data.get("elements", []):
        # Extrair C_Salesperson do JSON
        c_salesperson = contact.get("C_Salesperson", None)
        c_email = contact.get("C_EmailAddress", None)

        # Verificar se C_Salesperson é nulo ou vazio
        if c_salesperson is None or not c_salesperson.strip():
            print("C_Salesperson é nulo ou vazio. Passando para o próximo objeto.")
            continue

        search = c_salesperson
        search = search.replace(' ', '*')

        # Pesquisar na segunda API para obter o primeiro objeto dentro de fieldValues
        salesperson_api_url = f"https://secure.p04.eloqua.com/api/REST/1.0/data/customObject/1143?search=name='{search}*'"
        response = requests.get(salesperson_api_url, auth=auth)
        salesperson_data = response.json()

        # Verificar se a solicitação foi bem-sucedida
        if response.status_code != 200:
            print(
                f"Falha ao obter dados do salesperson. Código de status: {response.status_code}")
            print(f"Resposta: {response.text}")
            continue

        # Extrair informações do primeiro objeto dentro de fieldValues
        field_values = salesperson_data["elements"][0].get("fieldValues", [])
        if not field_values:
            print("Campo fieldValues não encontrado no JSON do salesperson.")
            continue

        # esse é o ID do lookup
        nome_do_campo_value = field_values[0]["value"]
        # Concatenando valores do terceiro e quarto campos
        nome_do_campo_key = f"{field_values[2]['value']} {field_values[3]['value']}"

        # Passo 1: Buscar o ID da lookup table
        eloqua_url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/lookupTables?search=name='{nome_lookup_table}'"
        response = requests.get(eloqua_url, auth=(auth))
        lookup_table_data = response.json()
        lookup_table_id = lookup_table_data["elements"][0]["id"]
        print(f"ID da lookup table {nome_lookup_table}: {lookup_table_id}")

        lookup_table_entries_url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/lookupTable/{lookup_table_id}/lookupTableEntries"
        page = 1
        total_pages = 1  # Inicializado com 1 para entrar no loop

        key_found = False  # Variável para indicar se a chave foi encontrada

        while page <= total_pages and not key_found:
            # Adiciona a lógica de paginação
            query_params = {"page": page}

            response = requests.get(
                lookup_table_entries_url, params=query_params, auth=(auth))

            if response.status_code == 200:
                entries_data = response.json()

                # Processa as entradas na página atual
                for entry in entries_data["elements"]:
                    key = entry.get("key")
                    if key == c_salesperson:
                        print(
                            f"A chave {c_salesperson} já existe na lookup table. O script não adicionará um novo valor.")
                        # Passo 1: Buscar o ID do contato
                        contact_id = get_contact_id(c_email, auth)

                        if contact_id:
                            # Passo 2: Atualizar as informações do contato
                            update_contact(contact_id, c_email,
                                           nome_do_campo_value, auth)
                        else:
                            print('Contato não encontrado.')
                        key_found = True  # Define a variável para indicar que a chave foi encontrada
                        break  # Pode interromper o loop se a chave for encontrada

                # Atualiza informações de paginação
                total_pages = (entries_data.get("total") //
                               entries_data.get("pageSize")) + 1
                page += 1

        # Passo 2: Adicionar um novo valor à lookup table, se a chave não foi encontrada
        if not key_found:
            lookup_table_entry_url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/lookupTable/{lookup_table_id}/lookupTableEntry"
            lookup_table_entry_data = {
                "type": "LookupTableEntry",
                "key": f"{c_salesperson}",
                "value": f"{nome_do_campo_value}"
            }

            headers = {
                "Content-Type": "application/json"
            }

            response = requests.post(lookup_table_entry_url, data=json.dumps(
                lookup_table_entry_data), headers=headers, auth=(auth))

            # Verificar se a operação foi bem-sucedida
            if response.status_code == 201:
                print(
                    f"Novo valor adicionado com sucesso à lookup table {nome_lookup_table}")
                # Passo 1: Buscar o ID do contato
                contact_id = get_contact_id(c_email, auth)

                if contact_id:
                    # Passo 2: Atualizar as informações do contato
                    update_contact(contact_id, c_email,
                                   nome_do_campo_value, auth)
                else:
                    print('Contato não encontrado.')
            else:
                print(
                    f"Erro ao adicionar valor à lookup table. Código de status: {response.status_code}")
                print(f"Resposta: {response.text}")


def update_country_eloqua(email, auth):
    # Etapa 1: Pesquisar por email
    search_url = f'https://secure.p04.eloqua.com/api/REST/2.0/data/contacts?search={email}'
    response = requests.get(search_url, auth=auth, headers={
                            'Content-Type': 'application/json'})

    # Verificar se a resposta foi bem-sucedida (código 200)
    if response.status_code == 200:
        # Obter o ID do primeiro objeto retornado
        contacts = response.json().get('elements', [])
        if contacts:
            contact_id = contacts[0]['id']

            # Etapa 2: Pesquisar por ID
            get_url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contact/{contact_id}'
            response = requests.get(get_url, auth=auth, headers={
                                    'Content-Type': 'application/json'})

            if response.status_code == 200:
                # Etapa 3: Atualizar o país para "US"
                put_url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contact/{contact_id}'
                payload = {
                    "type": "Contact",
                    "emailAddress": email,
                    "id": contact_id,
                    "country": "US"
                }
                response = requests.put(put_url, json=payload, auth=auth, headers={
                                        'Content-Type': 'application/json'})

                if response.status_code == 200:
                    return f"O país para o contato com ID {contact_id} foi atualizado para 'US'."
                else:
                    return f"Erro ao atualizar o país: {response.status_code}"
        else:
            return "Nenhum contato encontrado com o email fornecido."


def eloqua_api_update_contact_field(email, most_common_value_100100, auth):
    # Função para buscar o ID do Contato pelo E-mail
    def get_contact_id(email):
        url = f'https://secure.p04.eloqua.com/api/REST/2.0/data/contacts?search={email}'
        response = requests.get(url, auth=auth)

        if response.status_code == 200:
            contact_data = response.json().get('elements', [])
            if contact_data:
                return contact_data[0]['id']
        return None

    # Função para realizar a Operação de PUT
    def update_contact_field(contact_id, most_common_value_100100, email):
        url = f'https://secure.p04.eloqua.com/api/REST/1.0/data/contact/{contact_id}'
        headers = {'Content-Type': 'application/json'}
        data = {
            "type": "Contact",
            "id": contact_id,
            "emailAddress": email,
            "fieldValues": [
                {
                    "type": "FieldValue",
                    "id": "100317",
                    "value": most_common_value_100100
                }
            ]
        }
        response = requests.put(url, json=data, headers=headers, auth=auth)
        return response.status_code

    # Exemplo de uso
    contact_id = get_contact_id(email)

    if contact_id:
        status_code = update_contact_field(
            contact_id, most_common_value_100100, email)

        if status_code == 200:
            print(f'Atualizado!')
        else:
            print(
                f'Erro ao atualizar o contato. Código de status: {status_code}')
    else:
        print(f'Contato com o e-mail {email} não encontrado.')


def search_contact_by_email(base_url, auth, email):
    # dominio = email.split('@')[1]
    search_url = f"{base_url}/api/REST/1.0/data/contacts?search=name={email}&depth=complete"
    response = requests.get(search_url, auth=auth)
    return response


def verify_us_country(email, base_url, auth):
    response = search_contact_by_email(base_url, auth, email)
    if response.status_code == 200:
        data = response.json()
        if "elements" in data:
            elements = data["elements"]
            for element in elements:
                if "country" in element and element["country"] == "US":
                    return True

                # Coletar os valores do campo "country" de todos os contatos
                countries = [element.get(
                    "country") for element in elements if "country" in element]
                total_contacts = len(countries)
                us_count = countries.count("US")

                if total_contacts > 0:
                    if us_count/total_contacts >= 0.5:
                        print(colored(
                            f"Email: {email} - A maioria dos contatos tem 'country' igual a 'US'.", "green"))
                        # Chamada da função
                        update_country_eloqua(email, auth)
                    else:
                        print(colored(
                            f"Email: {email} - A maioria dos contatos tem 'country' diferente de 'US'.", "red"))
                else:
                    print(colored(
                        f"Email: {email} - Assuming 'US'.", "green"))
                    update_country_eloqua(email, auth)
    else:
        print(f"Email: {email} - Falha na busca de contatos por e-mail.")


def search_account_by_name(base_url, auth, search_account_name):
    account_search_url = f"{base_url}/api/REST/1.0/data/accounts?search=name='{search_account_name}*'&depth=complete"
    account_response = requests.get(account_search_url, auth=auth)
    return account_response


def find_most_common_account_name(elements, with_salesperson):
    account_names = [element.get("accountName") for element in elements if (
        with_salesperson and "salesPerson" in element) or (not with_salesperson)]
    return Counter(account_names).most_common(1)[0][0]


def verify_account(email, base_url, auth):
    response = search_contact_by_email(base_url, auth, email)

    if response.status_code == 200:
        data = response.json()
        if "elements" in data:
            elements = data["elements"]
            if elements:
                for element in elements:
                    if "fieldValues" in element:
                        fieldValues_data = element.get("fieldValues")
                        if isinstance(fieldValues_data, list):
                            for field_data in fieldValues_data:
                                if "id" in field_data and "value" in field_data:
                                    if field_data["id"] == "100317":
                                        return True

                        account_name_found = True
                        break

                if not account_name_found:
                    print(
                        f"Email: {email} - Nenhum contato encontrado com 'salesPerson'.")
                    search_account_name = find_most_common_account_name(
                        elements, with_salesperson=False)
                    print(f"Email: {email} - Sem Sales: {search_account_name}")
                else:
                    search_account_name = find_most_common_account_name(
                        elements, with_salesperson=True)
                    print(f"Email: {email} - com Sales: {search_account_name}")
                search_account_name = search_account_name.replace(",", "").replace(
                    ".", "").replace(" ", "*").replace("™", "").replace("&", "").replace("and", "").replace("**", "*")
                account_response = search_account_by_name(
                    base_url, auth, search_account_name)

                if account_response.status_code == 200:
                    account_data = account_response.json()
                    if "elements" in account_data:
                        account_elements = account_data["elements"]
                        if account_elements:
                            values_100464 = []

                            for account in account_elements:
                                field_values = account.get("fieldValues", [])
                                for field in field_values:
                                    if field.get("id") == "100464":
                                        values_100464.append(
                                            (float(field.get("value")), field_values))

                            if values_100464:
                                max_value_100464, max_value_field_values = max(
                                    values_100464, key=lambda x: x[0])
                                value_100100 = next((field.get(
                                    "value") for field in max_value_field_values if field.get("id") == "100100"), None)

                                #  value_100100 é o ID do Account
                                if value_100100:
                                    print(colored(
                                        f"Email: {email} - O valor do campo com id 100100 que está dentro do mesmo fieldValue é: {value_100100}", "green"))
                                    most_common_value_100100 = value_100100
                                    eloqua_api_update_contact_field(
                                        email, most_common_value_100100, auth)
                                else:
                                    print(
                                        f"Email: {email} - Nenhum valor encontrado para o id 100100 no mesmo fieldValue.")
                            else:
                                values_100100 = [
                                    field.get("value") for field in field_values if field.get("id") == "100100"]
                                most_common_value_100100 = Counter(values_100100).most_common(1)[
                                    0][0] if values_100100 else None

                                if most_common_value_100100:
                                    print(colored(
                                        f"Email: {email} - O valor do campo com id 100100 que mais se repete é: {most_common_value_100100}", "green"))
                                    eloqua_api_update_contact_field(
                                        email, most_common_value_100100, auth)
                                else:
                                    print(
                                        f"Email: {email} - Nenhum valor encontrado para os ids 100464 ou 100100 na conta.")


if __name__ == "__main__":

    df = pd.read_excel('../verify/emails.xlsx')
    emails = df['Email']

    # # ------------------------------------------------------------------------------
    base_url = "https://secure.p04.eloqua.com"
    username, password = load_env_variables()
    auth = authenticate(username, password)

    # ------------------------------------------------------------------------------
    # função importada do arquivo lookup.py
    update_lookup(auth)
    print("Iniciano o script CDO...")

    # Verify US Country for each email
    for email in emails:
        if "@gmail.com" not in email:
            verify_us_country(email, base_url, auth)

    # # Verify Account for each email
    for email in emails:
        if "@gmail.com" not in email:
            verify_account(email, base_url, auth)

Verificando lookup...


IndexError: list index out of range